In [1]:
import pandas as pd
import os
import warnings

In [ ]:
#This is an experimental notebook - no functions are defined, no DRY principles followed, and no clean code. The purpose of this is just to facilitate data access for rapid prototyping of the analytics team.

In [2]:
warnings.filterwarnings("ignore", message="Workbook contains no default style, apply openpyxl's default")

In [3]:
directory = '../DiabGr'
directory2 = '../ContrGr'

dataframes = {}
dataframes2 = {}

In [11]:
# DiabGr
for filename in os.listdir(directory):
    if filename.endswith('.xlsx'):
        file_path = os.path.join(directory, filename)
        
        dataframes[filename] = pd.read_excel(file_path, engine="openpyxl")

In [12]:
for filename in os.listdir(directory2):
    if filename.endswith('.xlsx'):
        file_path = os.path.join(directory2, filename)
        
        dataframes2[filename] = pd.read_excel(file_path, engine="openpyxl")

In [15]:
first_key = next(iter(dataframes))
test_df = dataframes[first_key]
#print(test_df)

In [16]:
#print(test_df.head())
#print(test_df.count())
#print(test_df.describe())
#missing = test_df.isnull().sum()
#print(missing)

#for df in dataframes:
#    print(dataframes[df].count())
#    print(dataframes[df].isnull().sum())

In [17]:
record_counts = []
person = 'P032T0'

for filename, dataframe in dataframes.items():
    num_rows = dataframe.shape[0]  # Number of rows in the current DataFrame
    record_counts.append(num_rows)
    #print(f"Number of rows in {filename}: {num_rows}")

    # foot/position
    if person in filename:
        print(filename[5:])

    # take diff btw record timestamps here

print(f"Total number of rows across all files: {sum(record_counts)}")
print(f"Lowest number of rows: {min(record_counts)}")
print(f"Average number of rows: {sum(record_counts)/len(record_counts)}")

01R.xlsx
02L.xlsx
02R.xlsx
01L.xlsx
Total number of rows across all files: 57911
Lowest number of rows: 210
Average number of rows: 681.3058823529411


In [28]:
total_rows = 0

for filename, dataframe in dataframes.items():
    num_rows = dataframe.shape[0]
    total_rows += num_rows
    print(f"Number of rows in {filename}: {num_rows}")

print(f"Total number of rows across all files: {total_rows}")

Number of rows in P001T01R.xlsx: 290
Number of rows in P016T01R.xlsx: 268
Number of rows in P025T02L.xlsx: 258
Number of rows in P094T01R.xlsx: 1912
Number of rows in P009T02R.xlsx: 282
Number of rows in P031T01R.xlsx: 318
Number of rows in P010T01L.xlsx: 282
Number of rows in P046T01R.xlsx: 2040
Number of rows in P042T01R.xlsx: 2043
Number of rows in P007T01L.xlsx: 294
Number of rows in P094T01L.xlsx: 1912
Number of rows in P024T01L.xlsx: 246
Number of rows in P013T01L.xlsx: 301
Number of rows in P031T02L.xlsx: 246
Number of rows in P046T01L.xlsx: 2040
Number of rows in P015T01R.xlsx: 330
Number of rows in P001T02L.xlsx: 234
Number of rows in P069T01L.xlsx: 1979
Number of rows in P045T02L.xlsx: 2017
Number of rows in P013T01R.xlsx: 301
Number of rows in P007T02L.xlsx: 210
Number of rows in P021T01L.xlsx: 300
Number of rows in P022T02L.xlsx: 272
Number of rows in P007T01R.xlsx: 294
Number of rows in P012T02L.xlsx: 234
Number of rows in P021T02R.xlsx: 270
Number of rows in P046T02R.xlsx

In [21]:
dataframes['P016T01R.xlsx'].columns

Index(['Unnamed: 0', 'Time', 'Date', 'MTK1.T', 'MTK2.T', 'MTK3.T', 'MTK4.T',
       'MTK5.T', 'D1.T', 'L.T', 'C.T', 'Env.T', 'MTK1.P', 'MTK2.P', 'MTK3.P',
       'MTK4.P', 'MTK5.P', 'D1.P', 'L.P', 'C.P'],
      dtype='object')

In [23]:
dataframes['P001T01R.xlsx']

,Unnamed: 0,Time,Date,MTK1.T,MTK2.T,MTK3.T,MTK4.T,MTK5.T,D1.T,L.T,C.T,Env.T,MTK1.P,MTK2.P,MTK3.P,MTK4.P,MTK5.P,D1.P,L.P,C.P
0,1,09:35:32,2014-02-05,23.665153,23.670404,23.798297,24.064021,23.860387,23.166001,24.453311,23.809233,23.084740,93.337682,103.830583,165.839884,131.216472,141.792939,103.714719,176.566740,151.150779
1,2,09:35:52,2014-02-05,23.677651,23.698930,23.837383,24.097838,23.885617,23.181379,24.513571,23.842472,23.115352,76.512637,103.046557,168.813242,133.087164,146.415524,102.775794,178.830630,153.250787
2,3,09:36:12,2014-02-05,23.690148,23.727457,23.876470,24.131656,23.910847,23.196756,24.573830,23.875711,23.145964,59.687592,102.262532,171.786599,134.957856,151.038109,101.836870,181.094520,155.350795
3,4,09:36:32,2014-02-05,23.703697,23.756147,23.914859,24.165223,23.935938,23.212191,24.630898,23.907588,23.174277,44.048767,101.476076,174.626373,136.696587,155.507469,100.900611,183.307349,157.149321
4,5,09:36:52,2014-02-05,23.717247,23.784837,23.953249,24.198790,23.961030,23.227626,24.687967,23.939466,23.202591,28.409943,100.689620,177.466146,138.435318,159.976829,99.964353,185.520178,158.947848
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
285,286,11:10:32,2014-02-05,29.607099,30.278363,31.159644,31.600000,31.200000,29.296618,31.277023,29.048858,28.699799,0.000000,0.000000,203.622234,141.159973,177.937743,99.271956,176.759097,184.109595
286,287,11:10:52,2014-02-05,29.626991,30.289771,31.173320,31.600000,31.200000,29.299721,31.267042,29.035416,28.726847,0.000000,0.000000,202.170586,139.545454,177.441332,100.716216,175.837205,177.518124
287,288,11:11:12,2014-02-05,29.646987,30.300658,31.187887,31.600000,31.200000,29.301943,31.256606,29.021065,28.755833,0.000000,0.000000,200.483442,137.865174,176.916941,102.215652,174.906069,170.615705
288,289,11:11:32,2014-02-05,29.666984,30.311544,31.202454,31.600000,31.200000,29.304165,31.246170,29.006713,28.784819,0.000000,0.000000,198.796298,136.184894,176.392549,103.715088,173.974933,163.713286
